# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [10]:
import sqlite3    # Python interface for SQLite databases

In [11]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [12]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
clients:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
villes:
  0: LIEU(TEXT)
  1: REGION(TEXT)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [13]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [14]:
%%sql SELECT *
FROM Master1 ORDER BY NV;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [15]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

In [16]:
%%sql SELECT *
FROM Master1
where NP is NULL;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
9,Chapelle Chambertin,11.9,1973,None,None,None,None,None
40,Richebourg,12,1943,90,None,None,None,None
54,Beaujolais Villages,10.5,1976,210,None,None,None,None
63,Coteaux du Tricastin,10,1978,136,None,None,None,None
66,Tavel,11,1968,124,None,None,None,None
89,Cotes de Provence,11,1969,980,None,None,None,None


In [17]:
%%sql SELECT DISTINCT COUNT(NP)
FROM Master1
where NP is NOT NULL;

 * sqlite:///wine.db
Done.


COUNT(NP)
204


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [18]:
%%sql SELECT *
FROM Master2
WHERE LIEU IS NOT NULL
ORDER BY REGION, LIEU
LIMIT 40;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
2,Julienas,11.3,1974,1983-12-25,LYON,2,48,Hardy,Thomas,moyen,AUVERGNE-RHÔNE-ALPES
9,Chapelle Chambertin,11.9,1973,1982-03-26,LYON,20,5,Audiberti,Jacques,petit,AUVERGNE-RHÔNE-ALPES
10,Beaujolais Villages,11.8,1979,1983-06-06,LYON,10,5,Audiberti,Jacques,petit,AUVERGNE-RHÔNE-ALPES
31,Monthelie,11.4,1968,1984-12-31,LYON,6,26,Diderot,Denis,moyen,AUVERGNE-RHÔNE-ALPES
31,Monthelie,11.4,1968,1978-12-25,LYON,100,28,Dostoievsky,Fiodor,gros,AUVERGNE-RHÔNE-ALPES
35,Auxey Duresses,11.3,1914,1981-12-25,LYON,80,30,La Fontaine,Jean de,petit,AUVERGNE-RHÔNE-ALPES
10,Beaujolais Villages,11.8,1979,1982-10-30,CHALON SUR SAONE,500,2,Artaud,Antonin,moyen,BOURGOGNE-FRANCHE-COMTÉ
40,Richebourg,12,1943,1982-04-02,DIJON,25,17,Camus,Albert,petit,BOURGOGNE-FRANCHE-COMTÉ
40,Richebourg,12,1943,1980-12-31,DIJON,12,19,Ciceron,None,gros,BOURGOGNE-FRANCHE-COMTÉ
3,Savigny les Beaunes,12.1,1978,1987-12-24,MERCUREY,5,49,Kafka,Franz,petit,BOURGOGNE-FRANCHE-COMTÉ


In [62]:
%%sql SELECT *
FROM Master2
WHERE MILL = 1980
ORDER BY NB
LIMIT 40;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
1,Mercurey,11.5,1980,1977-11-02,BORDEAUX,33,2,Artaud,Antonin,moyen,NOUVELLE-AQUITAINE
4,Mercurey,10.9,1980,1982-11-05,TOULOUSE,12,8,Aragon,Louis,gros,OCCITANIE
1,Mercurey,11.5,1980,2015-10-16,PARIS,1,44,Gide,Andre,petit,ÎLE-DE-FRANCE
4,Mercurey,10.9,1980,2015-10-16,PARIS,6,44,Gide,Andre,petit,ÎLE-DE-FRANCE
16,Meursault,12.1,1980,2015-10-16,PARIS,2,44,Gide,Andre,petit,ÎLE-DE-FRANCE
20,Cote de Brouilly,12.1,1980,2015-10-16,PARIS,3,44,Gide,Andre,petit,ÎLE-DE-FRANCE
26,Chateau Corton Grancey,None,1980,2015-10-16,PARIS,4,44,Gide,Andre,petit,ÎLE-DE-FRANCE
28,Volnay,11,1980,2015-10-16,PARIS,5,44,Gide,Andre,petit,ÎLE-DE-FRANCE
43,Fleurie,11.4,1980,2015-10-16,PARIS,7,44,Gide,Andre,petit,ÎLE-DE-FRANCE
74,Arbois,12,1980,2015-10-16,PARIS,8,44,Gide,Andre,petit,ÎLE-DE-FRANCE


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [19]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [20]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
clients:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
villes:
  0: LIEU(TEXT)
  1: REGION(TEXT)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [21]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [22]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [23]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [24]:
%%sql DROP TABLE IF EXISTS wines;

-- Create wines table
CREATE TABLE wines AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM wines LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [25]:
%%sql DROP TABLE IF EXISTS producers;

-- Create producers table
CREATE TABLE producers AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

SELECT *
FROM producers LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [26]:
%%sql DROP TABLE IF EXISTS production;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NP, NV, QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;

SELECT *
FROM production LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


#### Exercise 1.4

Create the new tables from Master2:

In [27]:
%%sql DROP TABLE IF EXISTS clients;

-- Create clients table
CREATE TABLE clients AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

SELECT *
FROM clients LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


In [28]:
%%sql DROP TABLE IF EXISTS sales;

-- Create sales table
CREATE TABLE sales AS
SELECT DISTINCT NV, DATES, LIEU, QTE, NB
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL;

SELECT *
FROM sales LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,DATES,LIEU,QTE,NB
1,1977-11-02,BORDEAUX,33,2
1,2015-10-16,PARIS,1,44
1,1983-12-31,RENNES,1,45
2,1983-12-25,LYON,2,48
3,1978-11-01,NICE,6,7
3,1987-12-24,MERCUREY,5,49
4,1982-11-05,TOULOUSE,12,8
4,2015-10-16,PARIS,6,44
4,1981-06-29,SENS,4,50
5,1984-11-04,MERCUREY,50,2


In [29]:
%%sql DROP TABLE IF EXISTS villes;

-- Create villes table
CREATE TABLE villes AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

SELECT *
FROM villes LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ
TOULOUSE,OCCITANIE
SENS,BOURGOGNE-FRANCHE-COMTÉ
LILLE,HAUTS-DE-FRANCE
NANCY,GRAND EST


In [30]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
clients:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
villes:
  0: LIEU(TEXT)
  1: REGION(TEXT)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [31]:
%%sql
SELECT DISTINCT type AS Type_of_buveurs FROM clients

 * sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [32]:
%%sql
SELECT CRU, REGION FROM production JOIN wines ON wines.NV=production.NV 
JOIN producers ON producers.NP=production.NP
    WHERE CRU IN ('Pommard', 'Brouilly')

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [33]:
%%sql
SELECT REGION FROM production JOIN wines ON wines.NV=production.NV 
    JOIN producers ON producers.NP=production.NP
    WHERE CRU = 'Pommard'
INTERSECT
SELECT REGION FROM production JOIN wines ON wines.NV=production.NV 
    JOIN producers ON producers.NP=production.NP
    WHERE CRU = 'Brouilly'

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [34]:
%%sql
SELECT CRU, MILL, SUM(sales.QTE) AS Number_of_wines_bought FROM sales JOIN wines ON wines.NV=sales.NV GROUP BY CRU,MILL

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [36]:
%%sql
SELECT DISTINCT NV FROM production GROUP BY NV HAVING COUNT(NP) >=3


 * sqlite:///wine.db
Done.


NV
12
43
45
54
67
69
74
78
79
87


#### Exercise 2.6

Which producers have not produced any wine?

In [52]:
%%sql
SELECT DISTINCT pr.NP, NV FROM producers pr LEFT JOIN production pd ON pr.NP=pd.NP  WHERE NV IS NULL ORDER BY pr.NP


 * sqlite:///wine.db
Done.


NP,NV
3,None
6,None
8,None
16,None
17,None
18,None
19,None
20,None
21,None
26,None


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [64]:
%%sql
SELECT DISTINCT clients.NB FROM sales JOIN clients ON sales.NB = clients.NB JOIN wines ON sales.NV = wines.NV 
WHERE MILL = 1980 ORDER BY clients.NB


 * sqlite:///wine.db
Done.


NB
2
8
44
45
50
61


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [65]:
%%sql
SELECT DISTINCT clients.NB FROM clients WHERE clients.NB NOT IN 
(SELECT DISTINCT clients.NB FROM sales JOIN clients ON sales.NB = clients.NB JOIN wines ON sales.NV = sales.NV 
WHERE MILL = 1980 ORDER BY clients.NB)


 * sqlite:///wine.db
Done.


NB
11
13
16
18
20
21
23
25
27
29


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [66]:
%%sql
SELECT DISTINCT sales.NB FROM sales WHERE sales.NB IN 
(SELECT DISTINCT clients.NB FROM sales JOIN clients ON sales.NB = clients.NB JOIN wines ON sales.NV = wines.NV 
WHERE MILL = 1980 ORDER BY clients.NB) GROUP BY sales.NB HAVING count(sales.NB) = 1


 * sqlite:///wine.db
Done.


NB
45
50


#### Exercise 2.10

List all wines from 1980

In [68]:
%%sql
SELECT * FROM WINES WHERE MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [75]:
%%sql
SELECT DISTINCT sales.NV FROM sales JOIN wines ON sales.NV = wines.NV 
WHERE MILL = 1980 AND NB=2


 * sqlite:///wine.db
Done.


NV
1


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [89]:
%%sql
SELECT sales.NB, COUNT(sales.NB) FROM sales JOIN wines ON sales.NV = wines.NV 
WHERE MILL = 1980 AND NB IN (SELECT DISTINCT clients.NB FROM sales JOIN clients ON sales.NB = clients.NB JOIN wines ON sales.NV = wines.NV 
WHERE MILL = 1980 ORDER BY clients.NB) GROUP BY sales.NB HAVING COUNT(sales.NB) = (SELECT COUNT(NV) FROM WINES WHERE MILL = 1980)

 * sqlite:///wine.db
Done.


NB,COUNT(sales.NB)
44,18


In [98]:
%%sql
SELECT DISTINCT B.NB FROM sales B JOIN wines ON B.NV = wines.NV 
WHERE NOT EXISTS (SELECT V.NV FROM sales V JOIN wines ON V.NV = wines.NV 
    WHERE MILL = 1980 AND NOT EXISTS (SELECT S.NB FROM sales S WHERE S.NB = B.NB AND S.NV = V.NV))

 * sqlite:///wine.db
Done.


NB
44
